In [1]:
import numpy as np, pandas as pd
import torch, os, logging

from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

from preprocessing import load_dataset, normalize

# Set up Python logging
logging.basicConfig(level=logging.ERROR)

# Create a log directory with a timestamp to keep different runs separate
logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(logdir, exist_ok=True)
writer = SummaryWriter(logdir)

pd.set_option('display.precision', 20)

# torch.manual_seed(1234)
torch.autograd.set_detect_anomaly(True)

if __name__ == "__main__":

    # Load Data (change path if needed)
    path = ["airFoil2D_SST_31.677_-0.742_2.926_5.236_5.651"]
    train_data, len_list = load_dataset(path, 
                                       n_random_sampling = 0
                                       )

    Uinf, alpha, gamma_1, gamma_2, gamma_3 = float(path[0].split('_')[2]), float(path[0].split('_')[3])*np.pi/180, float(path[0].split('_')[4]), float(path[0].split('_')[5]), float(path[0].split('_')[6])
    print(f"Uinf: {Uinf}, alpha: {alpha}")
    
    u_inlet, v_inlet = np.cos(alpha)*Uinf, np.sin(alpha)*Uinf
    
    df_train_input = pd.DataFrame(train_data[0].x_train, columns=["x", "y", "sdf", "x_n", "y_n"])
    df_train_target = pd.DataFrame(train_data[0].y_train, columns=["u", "v", "p", "nut"])
    df_train = pd.concat([df_train_input, df_train_target], axis=1) 

    df_box = df_train.iloc[len_list[0]:len_list[0]+len_list[1]+len_list[2],:]

    df = df_box[(df_box['x'] <= 4) & (df_box['x'] >= 1.5)]

    df_train_n, _ = normalize(df_train)
    mean_variance_dict = {column: {"mean": 0.0, "var": 1.0} for i, column in enumerate(df.columns)}

    df_aerofoil = pd.DataFrame(columns=df_train.columns)

    df_freestream_n = df_train_n.iloc[:len_list[0],:]
    df_aerofoil_n = df_train_n.iloc[len_list[0]:len_list[0]+len_list[1],:]
    df_box_n = df_train_n.iloc[len_list[0]:len_list[0]+len_list[1]+len_list[2],:]
        
    print("Datasets Loaded.")

100%|██████████| 1/1 [00:00<00:00,  4.29it/s]

Uinf: 31.677, alpha: -0.012950343049797924
Datasets Loaded.


In [2]:
from models.PINN_Phy_Bc import PINN_Phy_Bc
from plot import plot_predictions_vs_test, plot_test

# Train the model
model = PINN_Phy_Bc(df, mean_variance_dict, u_inlet, v_inlet, gamma_1, gamma_2, gamma_3)
if torch.cuda.device_count() > 1:
    print(f"Let's use {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)
    
# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

print(f"Started Training.")
model.train(501)
print(f"Finished Training.")

layers: [8, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 4]
Started Training.
Loading checkpoint 'path_to_checkpoint.pth'
Resuming from iteration 11
Iteration: 12
Iteration: 14
Iteration: 16
Iteration: 18
Iteration: 20
u_train_loss: 0.871860921382904
v_train_loss: 0.04487809166312218
p_train_loss: 4.109555721282959
nut_train_loss: 0.08208496868610382
f_u_loss: 0.2818072736263275
f_v_loss: 0.2861118018627167
ic_loss: 0.011679950170218945
Saved checkpoint to 'path_to_checkpoint.pth' at iteration 20
Iteration: 22
Iteration: 24
Iteration: 26
Iteration: 28
Iteration: 30
u_train_loss: 0.9267498254776001
v_train_loss: 0.04196577146649361
p_train_loss: 2.981478214263916
nut_train_loss: 0.003535201773047447
f_u_loss: 0.1320420205593109
f_v_loss: 0.13932178914546967
ic_loss: 0.003426274983212352
Saved checkpoint to 'path_to_checkpoint.pth' at iteration 30
Iteration: 32
Iteration: 34
Iteration: 36
Iteration: 38
Iteration: 40
u_train_loss: 0.935783863067627
v_train_loss: 0.04345570132136345
p_train_los

In [ ]:
# Prediction u_pred, v_pred, p_pred, nut_pred
u_pred, v_pred, p_pred, nut_pred = model.predict(df_box_n, u_inlet, v_inlet, gamma_1, gamma_2, gamma_3)

# Plotting
plot_predictions_vs_test(df_box_n['x'].astype(float).values.flatten(), df_box_n['y'].astype(float).values.flatten(), u_pred, df_box_n['u'], 'u', 'PINN_Phy_Bc')
plot_predictions_vs_test(df_box_n['x'].astype(float).values.flatten(), df_box_n['y'].astype(float).values.flatten(), v_pred, df_box_n['v'], 'v', 'PINN_Phy_Bc')
plot_predictions_vs_test(df_box_n['x'].astype(float).values.flatten(), df_box_n['y'].astype(float).values.flatten(), p_pred, df_box_n['p'], 'p', 'PINN_Phy_Bc')
plot_predictions_vs_test(df_box_n['x'].astype(float).values.flatten(), df_box_n['y'].astype(float).values.flatten(), nut_pred, df_box_n['nut'], 'nut', 'PINN_Phy_Bc')

TypeError: PINN_Phy_Bc.net_NS() missing 8 required positional arguments: 'x', 'y', 'u_inlet', 'v_inlet', 'sdf', 'gamma_1', 'gamma_2', and 'gamma_3'